## Custom Operation in TorchScript
In several situations, a network can have components that are unknown to TensorRT. 

In such cases, the network will fail to convert into TensorRT engine because of unknown ops.
- There are several prevelant methods to integrate custom ops in a TensorRT engine. 
-  First, one can write custom plugins ( C++/ CUDA ) and register these plugins explicitly in TensorRTs plugin registry.
- However, this method requires involved development and significant man-hours.
- Another, efficient but not ideal method is to break up models into chunks and convert each chunk independently into TensorRT engines. Subsequently, when a part of the model fails to convert to TensorRT due to novel ops, we can convert these chunks to Torchscript instead. 
- In summary, every piece of the model compatible with TensorRT is converted directly, whil the remaining parts of the model are converted to torch script and stored as serialized objects.
- Once all parts of the model are converted ( to either TRT or TorchScript), we stitch together these parts to give single output.
- Outputs of one model chunk, is fed as input to the next chunk of the model. This is referred to as stitching engines.
- Now, we will look at a simple example where a custom OP stored as torchscript earlier, is read and used in inference.

In [1]:
import torch

Now we will try to load a custom OPs called bev_pool_forward ( which is of course not a part of torch).
As expected that would throw errors.

In [2]:
torch.ops.my_ops.bev_pool_forward

RuntimeError: No such operator my_ops::bev_pool_forward

Now, we will load the pre-compiled library as explained earlier, to register this operation in torch.
This would allow us to use custom libraries inside torch/

In [3]:
torch.ops.load_library("/home/ubuntu/workstation/motorai/tensorrt_torchscript_demo/bev_pool/build/lib.linux-x86_64-cpython-38/bev_pool_forward.cpython-38-x86_64-linux-gnu.so")

In [4]:
torch.ops.my_ops.bev_pool_forward

<function torch._ops.my_ops.PyCapsule.bev_pool_forward>

Now, we will load the pre-compiled torchscript and perform inference to demonstrate how a network with a custom op, can be loaded in Python/C++ environment 
for inference.

In [5]:
bev_pool_network = torch.jit.load("/home/ubuntu/workstation/motorai/tensorrt_torchscript_demo/torchscript_engines/bev_pool.pt")

Now, we add our sample inputs to this network to get the expected output.

In [9]:
x = torch.load("../data/x.pt")
geom = torch.load("../data/geom.pt")

In [10]:
output = bev_pool_network(geom,x)

In [12]:
output.shape

torch.Size([1, 80, 360, 360])